# Extra Credit

For upto an additional 15 extra credit points on the midterm exam, you can write another function that applies additional criteria on top of what the midterm already did.  To earn the addition points, you need write a function called `summarize_results(patients)` to take the output of your  `process_people()` function to apply a new business rule and summarize the results.

* Your function should take the dictionary created by `process_people()` as it's one input parameter: `patients`.
* If the PRIEST probably is less than 10%, change the `hospital` and `address` to both have a value of `'Home'` indicating patients with a lower probability of an adverse outcome can just go home rather than be admitted to a hospital.
* Summarize those results and count how many patients are now being sent to each location.  Your result should have the following format:
```
{
    "Home": 123,
    "Hospital A": 10,
    "Hospital B": 20,
    "Hospital C": 30
}
```

As always, your function needs good docstrings; but does not require any doctests in this case.

In [28]:
import requests
import json

In [29]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    (str, int, int, int, int, int, float, str, str, str) -> float

    This function calculate the PRIEST score and return the 30-day probability of an outcome.

    >>> priest("Male", 82, 25, 91, 131, 99, 38.3, "Alert", "Air", "Unrestricted Normal Activity")
    0.59

    >>> priest("Female", 83, 28, 88, 140, 223, 40.0, "Confused or Not Alert", "Supplemental Oxygen", "Bed/chair bound, no self-care")
    0.99

    >>> priest("Unknown", 70, 8, 92, 90, 120, 36.5, "Alert", "Air", "Unrestricted Normal Activity") is None
    True
    """

    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    sex_score = 0 if sex == "female" else 1 if sex == "male" else None

    if 16 <= age <= 49:
        age_score = 0
    elif 50 <= age <= 65:
        age_score = 2
    elif 66 <= age <= 80:
        age_score = 3
    elif age > 80:
        age_score = 4
    else:
        age_score = None

    if respiratory_rate < 9:
        resp_rate_score = 3
    elif 9 <= respiratory_rate <= 11:
        resp_rate_score = 1
    elif 12 <= respiratory_rate <= 20:
        resp_rate_score = 0
    elif 21 <= respiratory_rate <= 24:
        resp_rate_score = 2
    elif respiratory_rate > 24:
        resp_rate_score = 3
    else:
        resp_rate_score = None

    if oxygen_saturation > 95:
        oxygen_score = 0
    elif 94 <= oxygen_saturation <= 95:
        oxygen_score = 1
    elif 92 <= oxygen_saturation <= 93:
        oxygen_score = 2
    elif oxygen_saturation < 92:
        oxygen_score = 3
    else:
        oxygen_score = None

    if heart_rate < 41:
        heart_rate_score = 3
    elif 41 <= heart_rate <= 50:
        heart_rate_score = 1
    elif 51 <= heart_rate <= 90:
        heart_rate_score = 0
    elif 91 <= heart_rate <= 110:
        heart_rate_score = 1
    elif 111 <= heart_rate <= 130:
        heart_rate_score = 2
    elif heart_rate > 130:
        heart_rate_score = 3
    else:
        heart_rate_score = None

    if systolic_bp < 91:
        bp_score = 3
    elif 91 <= systolic_bp <= 100:
        bp_score = 2
    elif 101 <= systolic_bp <= 110:
        bp_score = 1
    elif 111 <= systolic_bp <= 219:
        bp_score = 0
    elif systolic_bp > 219:
        bp_score = 3
    else:
        bp_score = None

    if temperature < 35.1:
        temp_score = 3
    elif 35.1 <= temperature <= 36.0:
        temp_score = 1
    elif 36.1 <= temperature <= 38.0:
        temp_score = 0
    elif 38.1 <= temperature <= 39.0:
        temp_score = 1
    elif temperature > 39.0:
        temp_score = 2
    else:
        temp_score = None

    alertness_score = 0 if alertness == "alert" else 3 if alertness == "confused or not alert" else None

    inspired_oxygen_score = 0 if inspired_oxygen == "air" else 2 if inspired_oxygen == "supplemental oxygen" else None

    if performance_status == "unrestricted normal activity":
        performance_status_score = 0
    elif performance_status == "limited strenuous activity, can do light activity":
        performance_status_score = 1
    elif performance_status == "limited activity, can self-care":
        performance_status_score = 2
    elif performance_status == "limited self-care":
        performance_status_score = 3
    elif performance_status == "bed/chair bound, no self-care":
        performance_status_score = 4
    else:
        performance_status_score = None

    # Checking for invalid input values
    if None in [sex_score, age_score, resp_rate_score, oxygen_score, heart_rate_score, bp_score, temp_score, alertness_score, inspired_oxygen_score, performance_status_score]:
        return None

    # Sum up PRIEST score
    priest_score = (
        sex_score
        + age_score
        + resp_rate_score
        + oxygen_score
        + heart_rate_score
        + bp_score
        + temp_score
        + alertness_score
        + inspired_oxygen_score
        + performance_status_score
    )

    # Finding the 30-day probability based on PRIEST score
    if priest_score >= 0 and priest_score <= 1:
        return 0.01
    elif priest_score >= 2 and priest_score <= 3:
        return 0.02
    elif 17 >= priest_score <= 25:
        return 0.59
    elif priest_score >= 26:
        return 0.99
    elif priest_score == 4:
        return 0.03
    elif priest_score == 5:
        return 0.09
    elif priest_score == 6:
        return 0.15
    elif priest_score == 7:
        return 0.18
    elif priest_score == 8:
        return 0.22
    elif priest_score == 9:
        return 0.26
    elif priest_score == 10:
        return 0.29
    elif priest_score == 11:
        return 0.34
    elif priest_score == 12:
        return 0.38
    elif priest_score == 13:
        return 0.46
    elif priest_score == 14:
        return 0.47
    elif priest_score == 15:
        return 0.49
    elif priest_score == 16:
        return 0.55
    else:
        return None

In [30]:
def find_hospital(age, sex, risk):
    """(int, str, float) -> str
    This function calls a REST web service to determine the hospital for a patient based on age, sex, and risk.

    >>> find_hospital(43, 'male', 0.4)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(72, 'female', 0.02)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(40, 'male', 0.3)
    'Southwest Hospital and Medical Center'
    """
    # Base URL for the REST web service
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"

    # Prepare extended URL by appending parameters in query parameters
    url = f"{base_url}?age={age}&sex={sex}&risk_pct={risk}"

    try:
        # HTTP GET request
        response = requests.get(url)

        # Validate if the request was successful by checking the status code
        if response.status_code == 200:
            data = response.json()
            return data.get("hospital")
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

In [31]:
def get_address(hospital_name):
    """(str) -> str
    This function retrieves the address of a hospital in Atlanta region in Georgia based on its name.

    Example:
    >>> get_address("WELLSTAR SPALDING REGIONAL HOSPITAL")
    '601 SOUTH 8TH STREET'

    >>> get_address("LOUIS SMITH MEMORIAL HOSPITAL")
    '116 WEST THIGPEN AVENUE'

    >>> get_address("CHILDRENS HOSPITAL NAVICENT HEALTH")
    '888 PINE STREET'
    """
    # URL for the JSON file containing hospital data
    json_url = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    try:
        # Fetch the JSON file
        response = requests.get(json_url)

        if response.status_code == 200:
            hospitals_data = json.loads(response.text)

            # Look for the hospital by its name
            hospital_data = hospitals_data.get(hospital_name)

            # If the hospital is found then return its address
            if hospital_data:
                return hospital_data.get("ADDRESS")

            return None
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

In [32]:

def process_people(file_location):
    """(str) -> json
    Process patient data from a pipe-delimited file and return a list of dictionaries.
    """

    response = requests.get(file_location)

    if response.status_code != 200:
        return []

    data = response.text.splitlines()[1:]
    result = []

    for line in data:
        # Split the line into fields
        fields = line.strip().split('|')

        if len(fields) >= 11:
            patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = fields[:11]
            risk = priest(sex, int(age), int(breath), int(float(o2sat) * 100), int(heart), int(systolic), float(temp), alertness, inspired, status)
            hospital = find_hospital(age, sex.lower(), risk)
            address = get_address(hospital.upper())
            patient_data = {
                patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]
            }
            result.append(patient_data)

    return result

In [33]:
def summarize_results(patients):
    """(list) -> dict
    Process patient data from a pipe-delimited file and return a list of dictionaries.
    """
    result = []
    occurrence_count = {}

    for patient in patients:
        for patient_number, data in patient.items():
            [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address] = data
            risk = priest(sex, int(age), int(breath), int(float(o2sat) * 100), int(heart), int(systolic), float(temp), alertness, inspired, status)

            if risk < 0.1:
                hospital = 'Home'
                address = 'Home'

            patient_data = {
                patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]
            }

            result.append(patient_data)



            for data_dict in result:
                if len(data_dict) == 1:
                    value_list = list(data_dict.values())[0]
                    if len(value_list) >= 11:
                        element_11 = value_list[10]
                        occurrence_count[element_11] = occurrence_count.get(element_11, 0) + 1




    return occurrence_count

In [34]:
print(summarize_results(process_people("https://hds5210-data.s3.amazonaws.com/people.psv")))

{'Emory Dunwoody Medical Center': 3454, 'Wesley Woods Geriatric Hospital': 1002, 'Hughes Spalding Childrens Hospital': 245, 'Home': 249}
